In [14]:
import pandas as pd
from analysis import *
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import binom_test

In [15]:
surge_annotations = data.surge_evaluation.annotation_dataframe()
surge_annotations_comparative = data.surge_evaluation.comparative_annotation_dataframe()

surge_annotations

0  \
bot            category        label        item                               
rerank_blender likert dialogue emotional    (109,13)_rerank_blender        4   
                               consistent   (109,13)_rerank_blender        4   
                               grammatical  (109,13)_rerank_blender        4   
                               informative  (109,13)_rerank_blender        4   
                               proactive    (109,13)_rerank_blender        4   
...                                                                       ..   
               behavior        follow up    ((438,26)_rerank_blender, 14)  0   
                               topic switch ((438,26)_rerank_blender, 14)  0   
                               ignore       ((438,26)_rerank_blender, 14)  0   
                               irrelevant   ((438,26)_rerank_blender, 14)  0   
                               antisocial   ((438,26)_rerank_blender, 14)  0   

                                                                             1  \
bot            category        label        item                                 
rerank_blender likert dialogue emotional    (109,13)_rerank_blender        4.0   
                               consistent   (109,13)_rerank_blender        4.0   
                               grammatical  (109,13)_rerank_blender        3.0   
                               informative  (109,13)_rerank_blender        4.0   
                               proactive    (109,13)_rerank_blender        3.0   
...                                                                        ...   
               behavior        follow up    ((438,26)_rerank_blender, 14)  NaN   
                               topic switch ((438,26)_rerank_blender, 14)  NaN   
                               ignore       ((438,26)_rerank_blender, 14)  NaN   
                               irrelevant   ((438,26)_rerank_blender, 14)  NaN   
                               antisocial   ((438,26)_rerank_blender, 14)  NaN   

                                                                            2  
bot            category        label        item                               
rerank_blender likert dialogue emotional    (109,13)_rerank_blender       NaN  
                               consistent   (109,13)_rerank_blender       NaN  
                               grammatical  (109,13)_rerank_blender       NaN  
                               informative  (109,13)_rerank_blender       NaN  
                               proactive    (109,13)_rerank_blender       NaN  
...                                                                        ..  
               behavior        follow up    ((438,26)_rerank_blender, 14) NaN  
                               topic switch ((438,26)_rerank_blender, 14) NaN  
                               ignore       ((438,26)_rerank_blender, 14) NaN  
                               irrelevant   ((438,26)_rerank_blender, 14) NaN  
                               antisocial   ((438,26)_rerank_blender, 14) NaN  

[151664 rows x 3 columns]

# 8 Comprehensive Analysis

### Metric Sensitivity

In [16]:
from itertools import combinations
from scipy.stats import ttest_ind

def p_vals(df: pd.DataFrame, test='t'):
    """
    :param df: (bot, data point) x 1 -> score
    :param test: statistical test function (t for t test, p for prop test, s for sign test)
    :return: p values of test on each bot pair (pd.Series)
    """
    bots = set(df.index.get_level_values(0))
    bot_pairs = list(combinations(bots, 2))
    result = {}
    for ba, bb in bot_pairs:
        a = df.xs(ba).to_numpy().squeeze()
        b = df.xs(bb).to_numpy().squeeze()
        if test == 't':
            t, p = ttest_ind(a, b, equal_var=False)
        elif test == 'p':
            z, p = proportions_ztest(count=[
                sum(a), sum(b)
            ], nobs=[
                len(a), len(b)
            ])
        elif test == 's':
            # sign test
            a = a[a==1]
            b = b[b==1]
            p = binom_test(sum(a), sum(a)+sum(b), p=0.5)
        else:
            raise ValueError('invalid arg for param "test"')
        result[(ba, bb)] = p
    result_series = pd.Series(result.values(), result)
    return result_series

@to_file
def t_test_p_values_comparing_bots(annotations):
    annotations = get_singly_annotated(annotations)
    prop_annotations = annotations.xs(
        category.behavior, level=sym.category, drop_level=False
    )
    comp_annotations = annotations.xs(
        category.comparative, level=sym.category, drop_level=False
    )
    mean_annotations = annotations.drop(
        index=category.behavior, level=sym.category
    ).drop(
        index=category.comparative, level=sym.category
    )
    mean_ps = mean_annotations.groupby(
        [sym.category, sym.label]
    ).apply(p_vals)
    prop_ps = prop_annotations.groupby(
        [sym.category, sym.label]
    ).apply(lambda x: p_vals(x, test='p'))
    comp_ps = comp_annotations.groupby(
        [sym.category, sym.label]
    ).apply(lambda x: p_vals(x, test='s'))
    result = pd.concat([prop_ps, mean_ps, comp_ps], axis=0)
    return result

t_test_p_values_comparing_bots(surge_annotations, reload='results/t_test_p_values_comparing_bots')

rerank_blender                \
                                                   emora   blender2_3B   
category        label                                                    
behavior        antisocial                  2.474602e-03  4.194563e-01   
                commonsense contradiction   4.674101e-08  1.014559e-01   
                correct fact                7.162630e-16  6.660799e-01   
                empathetic                  3.697819e-01  3.927420e-01   
                follow up                   2.214653e-01  2.639515e-26   
                ignore                      1.197645e-03  3.957589e-01   
                incorrect fact              1.851676e-62  9.507153e-55   
                irrelevant                  1.449497e-01  4.882563e-09   
                lack of empathy             1.615134e-05  6.580446e-04   
                life info                   1.880903e-02  1.455560e-01   
                partner contradiction       8.959598e-01  1.730739e-01   
                preference info             4.708673e-04  2.621806e-05   
                redundant                   2.507068e-02  4.701412e-06   
                self contradiction          3.031944e-11  1.744282e-02   
                topic switch                9.414420e-02  7.325721e-12   
                uninterpretable             1.566895e-08  2.259988e-07   
likert dialogue consistent                  2.748855e-01  7.327280e-02   
                emotional                   1.658327e-01  6.041001e-05   
                engaging                    3.156970e-01  5.314064e-01   
                grammatical                 2.151765e-06  6.783573e-08   
                informative                 7.671632e-04  1.808310e-02   
                proactive                   1.653928e-01  6.589875e-01   
                quality                     2.152175e-01  1.150702e-02   
                relevant                    3.673116e-01  4.738332e-03   
likert turn     consistent                  2.753539e-02  1.775618e-01   
                emotional                   2.565909e-09  3.801480e-01   
                engaging                    5.943432e-02  2.268474e-03   
                grammatical                 4.112589e-62  4.328568e-82   
                informative                 1.636008e-39  1.723153e-14   
                proactive                   3.053150e-03  7.148207e-02   
                quality                     2.306632e-06  1.169420e-28   
                relevant                    8.985054e-02  5.131697e-26   
comparative     consistent                  4.074855e-01  5.577833e-02   
                emotional                   1.000000e+00  4.119101e-01   
                engaging                    1.597615e-01  4.280722e-02   
                grammatical                 5.764306e-01  9.151865e-01   
                informative                 4.087197e-02  1.424496e-03   
                proactive                   4.596914e-01  1.014327e-01   
                quality                     8.438977e-01  4.167748e-01   
                relevant                    9.219488e-01  5.383940e-01   

                                                                   emora  \
                                          bart_fid_rag_bcb   blender2_3B   
category        label                                                      
behavior        antisocial                    1.087556e-02  1.427387e-02   
                commonsense contradiction     2.194954e-02  1.139109e-04   
                correct fact                  9.406698e-19  1.753072e-17   
                empathetic                    5.085360e-10  9.659565e-01   
                follow up                     4.876607e-32  2.111994e-32   
                ignore                        5.297479e-03  1.598324e-02   
                incorrect fact                1.503432e-01  1.317827e-03   
                irrelevant                    9.615384e-07  9.402470e-06   
                lack of

### Predictive Validity

In [17]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
from statsmodels.regression.linear_model import OLS as LinearModel
from statsmodels.tools.tools import add_constant

def dialogue_metrics(ev):
    df: pd.DataFrame = ev.annotation_dataframe()
    df = get_singly_annotated(df, seed=123)
    reindexed = df.reset_index()
    items = reindexed[sym.item]
    dialogues = [e[0] if isinstance(e, tuple) else e for e in items]
    reindexed['dialogue'] = dialogues
    reindexed.set_index(
        [sym.bot, sym.category, sym.label, 'dialogue', sym.item],
        inplace=True, verify_integrity=True
    )
    ld = reindexed.xs(category.likert_dialogue, level=sym.category)
    ld = ld.droplevel(sym.bot).droplevel(sym.item)
    ld.columns = ['score']
    ldq = ld.xs(scale.quality, level=sym.label)
    ldq.columns = ['quality']

    lt = reindexed.xs(category.likert_turn, level=sym.category)
    lt = lt.groupby([sym.label, 'dialogue']).mean()
    lt.columns = ['score']
    ltq = lt.xs(scale.quality, level=sym.label)
    ltq.columns = ['quality']

    be = reindexed.xs(category.behavior, level=sym.category)
    be = be.groupby([sym.label, 'dialogue']).mean()
    be.columns = ['score']

    ds = pd.concat(
        [lt, be, ld],
        keys=[category.likert_turn, category.behavior, category.likert_dialogue],
        names=[sym.category, sym.label, 'dialogue']
    )
    likert_dialogue_quality_features = ds.join(ldq, on='dialogue')
    likert_turn_quality_features = ds.join(ltq, on='dialogue')
    return likert_dialogue_quality_features, likert_turn_quality_features


def regressions(df, quality_column_name=None, model='linear'):
    """
    :param df: dialogue x (*features, quality) -> value
    :return: *(coef, low, high), mcfadden r^2
    """
    if not quality_column_name:
        quality_column_name = df.columns[-1]
    qualities = df[quality_column_name]
    features = [f for f in df.columns if f != quality_column_name]
    if model == 'ordinal':
        model = OrderedModel(qualities, df[features], distr='logit')
        results = model.fit()
        coefs = {f: results.params[f] for f in features}
        prsqrd = results.prsquared
        result = {stat.mcfad_r2: prsqrd, stat.p_of_llr_test: results.llr_pvalue}
    elif model == 'linear':
        x = add_constant(df[features])
        y = qualities
        model = LinearModel(y, x)
        results = model.fit()
        coefs = {f: results.params[f] for f in features}
        rsquared = results.rsquared
        result = {**coefs, stat.r2: rsquared, stat.p_of_f_test: results.f_pvalue}
    else:
        raise ValueError('Param "model" must be one of {"linear", "ordinal"}')
    return pd.Series(result.values(), result)

In [18]:
@to_file
def dialogue_quality_regressions(ev):
    ldq, ltq = dialogue_metrics(ev)
    ldq_groups = ldq.groupby(
        [sym.category, sym.label]
    )
    ltq_groups = ltq.groupby(
        [sym.category, sym.label]
    )
    names = ['Predicted', 'Metric']
    linear_result = ldq_groups.apply(lambda x: regressions(x, model='linear'))
    linear_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Dialogue Quality']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    ordinal_result = ldq_groups.apply(lambda x: regressions(x, model='ordinal'))
    ordinal_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Dialogue Quality']*2,
        ['OR Pseudo R-Squared', stat.p_of_llr_test]],
        names=names
    )
    linear_turn_result = ltq_groups.apply(regressions)
    linear_turn_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Turn Quality']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    result = pd.concat((linear_turn_result, linear_result, ordinal_result), axis=1)
    return result.round(5)

regs = dialogue_quality_regressions(
    data.surge_evaluation,
    reload='results/dialogue_quality_regressions'
)
regs

Optimization terminated successfully.
         Current function value: 1.314613
         Iterations: 217
         Function evaluations: 355
Optimization terminated successfully.
         Current function value: 1.314321
         Iterations: 193
         Function evaluations: 320
Optimization terminated successfully.
         Current function value: 1.314382
         Iterations: 210
         Function evaluations: 341
Optimization terminated successfully.
         Current function value: 1.327590
         Iterations: 118
         Function evaluations: 207
Optimization terminated successfully.
         Current function value: 1.327138
         Iterations: 188
         Function evaluations: 311
Optimization terminated successfully.
         Current function value: 1.314526
         Iterations: 222
         Function evaluations: 360
Optimization terminated successfully.
         Current function value: 1.313996
         Iterations: 212
         Function evaluations: 354
Optimization termina

/Users/sarah/opt/anaconda3/envs/dia_analysis/lib/python3.10/site-packages/statsmodels/base/model.py:592: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  warnings.warn('Inverting hessian failed, no bse or cov_params '


Predicted                                 Likert Turn Quality               \
Metric                                         LR Coefficient LR R-Squared   
category        label                                                        
likert turn     consistent                            0.21707      0.04474   
                emotional                             0.18738      0.03755   
                engaging                              0.16420      0.04061   
                grammatical                           0.29094      0.06348   
                informative                           0.02049      0.00046   
                proactive                             0.22061      0.06614   
                quality                               1.00000      1.00000   
                relevant                              0.26306      0.09870   
behavior        antisocial                           -3.70553      0.01164   
                commonsense contradiction            -1.29550      0.07139   
                correct fact                         -0.34247      0.00899   
                empathetic                            0.45030      0.02107   
                follow up                             0.46015      0.02544   
                ignore                               -1.74979      0.07209   
                incorrect fact                       -0.83946      0.03412   
                irrelevant                           -1.26057      0.06118   
                lack of empathy                      -1.16185      0.04956   
                life info                             0.18006      0.00186   
                partner contradiction                -1.49766      0.04030   
                preference info                       0.31426      0.00875   
                redundant                            -0.42368      0.00251   
                self contradiction                   -1.27412      0.02447   
                topic switch                         -0.56434      0.01100   
                uninterpretable                      -1.59035      0.00523   
likert dialogue consistent                            0.03652      0.00544   
                emotional                             0.11540      0.03715   
                engaging                              0.06870      0.01141   
                grammatical                           0.02164      0.00097   
                informative                           0.01384      0.00038   
                proactive                             0.05628      0.00901   
                quality                               0.12029      0.02906   
                relevant                              0.11919      0.03677   

Predicted                                                    \
Metric                                    P value of F-test   
category        label                                         
likert turn     consistent                          0.00002   
                emotional                           0.00010   
                engaging                            0.00005   
                grammatical                         0.00000   
                informative                         0.66881   
                proactive                           0.00000   
                quality                             0.00000   
                relevant                            0.00000   
behavior        antisocial                          0.03100   
                commonsense contradiction           0.00000   
                correct fact                        0.05813   
                empathetic                          0.00362   
                follow up                           0.00137   
                ignore                              0.00000   
                incorrect fact                      0.00020   
                irrelevant                          0.00000   
                lack of empathy                     0.00001   
         

In [19]:
to_plot_regs = regs[[("Likert Dialogue Quality", "LR R-Squared"), ("Likert Dialogue Quality", "P value of F-test")]]
to_plot_regs = to_plot_regs.drop(("likert dialogue", "quality"))
to_plot_regs = to_plot_regs.reset_index()
to_plot_regs

Predicted         category                      label Likert Dialogue Quality  \
Metric                                                           LR R-Squared   
0              likert turn                 consistent                 0.02999   
1              likert turn                  emotional                 0.02943   
2              likert turn                   engaging                 0.02729   
3              likert turn                grammatical                 0.00010   
4              likert turn                informative                 0.00159   
5              likert turn                  proactive                 0.02868   
6              likert turn                    quality                 0.02906   
7              likert turn                   relevant                 0.03776   
8                 behavior                 antisocial                 0.00013   
9                 behavior  commonsense contradiction                 0.06103   
10                behavior               correct fact                 0.00010   
11                behavior                 empathetic                 0.04044   
12                behavior                  follow up                 0.00440   
13                behavior                     ignore                 0.03455   
14                behavior             incorrect fact                 0.01915   
15                behavior                 irrelevant                 0.04943   
16                behavior            lack of empathy                 0.04384   
17                behavior                  life info                 0.00306   
18                behavior      partner contradiction                 0.06043   
19                behavior            preference info                 0.00306   
20                behavior                  redundant                 0.02931   
21                behavior         self contradiction                 0.02745   
22                behavior               topic switch                 0.00001   
23                behavior            uninterpretable                 0.00772   
24         likert dialogue                 consistent                 0.22558   
25         likert dialogue                  emotional                 0.23022   
26         likert dialogue                   engaging                 0.38103   
27         likert dialogue                grammatical                 0.07243   
28         likert dialogue                informative                 0.17475   
29         likert dialogue                  proactive                 0.27452   
30         likert dialogue                   relevant                 0.28720   

Predicted                    
Metric    P value of F-test  
0                   0.00050  
1                   0.00057  
2                   0.00091  
3                   0.83866  
4                   0.42573  
5                   0.00067  
6                   0.00062  
7                   0.00009  
8                   0.82001  
9                   0.00000  
10                  0.84188  
11                  0.00005  
12                  0.18548  
13                  0.00019  
14                  0.00557  
15                  0.00001  
16                  0.00002  
17                  0.26971  
18                  0.00000  
19                  0.26993  
20                  0.00058  
21                  0.00088  
22                  0.94880  
23                  0.07922  
24                  0.00000  
25                  0.00000  
26                  0.00000  
27                  0.00000  
28                  0.00000  
29                  0.00000  
30                  0.00000

In [20]:
regs = prettify(regs, to_csv="results/paper/predictive_validity.csv")

# Incremental Validity

In [21]:
def drop_column_level_duplication(df: pd.DataFrame, columns, levels=None):
    if levels is None:
        levels = list(range(len(columns)))
    level_columns = df.xs(columns, axis=1, level=levels)
    unique = level_columns.iloc[:,0].to_frame()
    unique.columns = [columns]
    dropped = df.drop(columns=columns, level=levels)
    result = pd.concat([dropped, unique], axis=1)
    return result

def multivariate_regression(df: pd.DataFrame, model='linear'):
    def apply_regressions(df: pd.DataFrame):
        unstacked = df.unstack([sym.category, sym.label])
        unstacked = drop_column_level_duplication(unstacked, 'quality', 0)
        results = regressions(unstacked, quality_column_name='quality', model=model)
        return results
    result = apply_regressions(df)
    result.index = [
        (idx[1] if isinstance(idx, tuple) else idx)
        for idx in result.index.values
    ]
    return result.round(5)

from collections import namedtuple

@to_file
def incremental_regression(
        df: pd.DataFrame,
        categories,
        model='linear',
        exclude_quality=True,
        beam=1,
        select='backward'
):
    data_points = set(df.index.get_level_values('dialogue'))
    num_data_points = len(data_points)
    adjust = lambda r2, f: 1 - (1 - r2) * ((num_data_points - 1) / (num_data_points - f))
    Step: type = namedtuple('Step', ('r2', 'p', 'feature'))
    class Path(list):
        def metric(self):
            # if len(self) == 0: return 0
            # else: return self[-1].llr if len(self) == 1 else self[-1].llr / self[-2].llr
            return self.r2
        @property
        def r2(self):
            return adjust(self[-1].r2, len(self)) if self else 0
        # @property
        # def adj_r2(self):
        #     return adjust(self.r2, len(self))
        @property
        def p(self): return self[-1].p if self else 1
        @property
        def features(self): return {x.feature for x in self}
    r2_name = stat.r2 if model=='linear' else stat.mcfad_r2
    p_name = stat.p_of_f_test if model=='linear' else stat.p_of_llr_test
    frontier = [Path()]
    feature_pool = {
        x[:2] for x in df.index.values
        if (not (exclude_quality and scale.quality in x))
        and x[0] in categories
    }
    for _ in feature_pool:
        new_frontier = []
        for path in frontier:
            for candidate in feature_pool - path.features:
                if select == 'forward':
                    candidate_features = path.features | {candidate}
                elif select == 'backward':
                    candidate_features = feature_pool - path.features
                else:
                    raise ValueError('param select must be one of {"forward", "backward"}')
                row_mask = [
                    x[:2] in candidate_features
                    and (not (exclude_quality and scale.quality in x))
                    and x[0] in categories
                    for x in df.index.values
                ]
                candidate_df = df.loc[row_mask, :]
                candidate_results = multivariate_regression(candidate_df, model=model)
                r2 = candidate_results[r2_name].item()
                p = candidate_results[p_name]
                new_frontier.append(Path([*path, Step(r2, p, candidate)]))
        frontier = sorted(new_frontier, key=lambda x: x.metric(), reverse=True)[:beam]
    result = {step.feature: {'Adjusted R-Squared': step.r2, p_name: step.p} for step in frontier[0]}
    return pd.DataFrame(result.values(), result)


ldq, ltq = dialogue_metrics(data.surge_evaluation)
regs = incremental_regression(
    ldq, (category.likert_turn, category.behavior), beam=1,
    reload='results/dialogue_incremental_regressions'
)
regs

Adjusted R-Squared  P value of F-test
likert turn emotional                             0.17799            0.00000
            relevant                              0.17720            0.00000
            consistent                            0.17213            0.00000
behavior    redundant                             0.17160            0.00000
            empathetic                            0.16777            0.00000
            commonsense contradiction             0.15683            0.00000
likert turn informative                           0.15412            0.00000
behavior    self contradiction                    0.15336            0.00000
            life info                             0.14527            0.00000
likert turn proactive                             0.14519            0.00000
behavior    antisocial                            0.13908            0.00000
            incorrect fact                        0.13901            0.00000
            partner contradiction                 0.12782            0.00000
            correct fact                          0.11063            0.00000
            lack of empathy                       0.10907            0.00000
            irrelevant                            0.10015            0.00000
likert turn engaging                              0.07098            0.00014
behavior    uninterpretable                       0.04991            0.00253
            topic switch                          0.04041            0.00600
            follow up                             0.03633            0.00547
likert turn grammatical                           0.03606            0.00223
behavior    ignore                                0.03606            0.00068
            preference info                       0.00306            0.26993

In [22]:
behavior_regs = incremental_regression(
    ldq, (category.behavior,), beam=10,
    reload='results/behavior_incremental_regressions'
)
behavior_regs

Adjusted R-Squared  P value of F-test
behavior antisocial                            0.15964                0.0
         life info                             0.15955                0.0
         preference info                       0.15850                0.0
         commonsense contradiction             0.15692                0.0
         uninterpretable                       0.15503                0.0
         ignore                                0.15250                0.0
         correct fact                          0.15197                0.0
         topic switch                          0.14809                0.0
         follow up                             0.14527                0.0
         redundant                             0.14366                0.0
         lack of empathy                       0.13925                0.0
         incorrect fact                        0.13547                0.0
         self contradiction                    0.12608                0.0
         irrelevant                            0.11049                0.0
         empathetic                            0.09489                0.0
         partner contradiction                 0.06043                0.0

In [23]:
likert_turn_regs = incremental_regression(
    ldq, (category.likert_turn,), beam=10,
    reload='results/likert_turn_incremental_regressions'
)
likert_turn_regs

Adjusted R-Squared  P value of F-test
likert turn grammatical             0.08566            0.00001
            informative             0.08508            0.00000
            engaging                0.08285            0.00000
            emotional               0.07753            0.00000
            consistent              0.06889            0.00000
            proactive               0.05786            0.00001
            relevant                0.03776            0.00009

In [24]:
likert_dialogue_regs = incremental_regression(
    ldq, (category.likert_dialogue,), beam=10,
    reload='results/likert_dialogue_incremental_regressions'
)
likert_dialogue_regs

Adjusted R-Squared  P value of F-test
likert dialogue emotional               0.58426                0.0
                informative             0.57832                0.0
                grammatical             0.56439                0.0
                relevant                0.54517                0.0
                proactive               0.51636                0.0
                consistent              0.48673                0.0
                engaging                0.38103                0.0

## Table for Paper

In [25]:
def add_delta(df):
    df['R-Squared'] = (df['Adjusted R-Squared']*100)
    df['delta'] = df['R-Squared'].diff()
    df['delta'] = df['delta'].map('{:.4f}'.format)
    df['R-Squared'] = df['R-Squared'].map('{:.4f}'.format)
    df['R-Squared delta'] = df['R-Squared'] + ' (' + df['delta'] + ')'

final_behavior_regs = behavior_regs.reset_index().rename({'level_1': 'ABC-Eval'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_behavior_regs)

final_likert_turn_regs = likert_turn_regs.reset_index().rename({'level_1': 'Likert Turn'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_likert_turn_regs)

final_likert_dialogue_regs = likert_dialogue_regs.reset_index().rename({'level_1': 'Likert Dialogue'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_likert_dialogue_regs)

final_behavior_regs

,ABC-Eval,Adjusted R-Squared,R-Squared,delta,R-Squared delta
0,antisocial,0.15964,15.9640,nan,15.9640 (nan)
1,life info,0.15955,15.9550,-0.0090,15.9550 (-0.0090)
2,preference info,0.15850,15.8500,-0.1050,15.8500 (-0.1050)
3,commonsense contradiction,0.15692,15.6920,-0.1580,15.6920 (-0.1580)
4,uninterpretable,0.15503,15.5030,-0.1890,15.5030 (-0.1890)
5,ignore,0.15250,15.2500,-0.2530,15.2500 (-0.2530)
6,correct fact,0.15197,15.1970,-0.0530,15.1970 (-0.0530)
7,topic switch,0.14809,14.8090,-0.3880,14.8090 (-0.3880)
8,follow up,0.14527,14.5270,-0.2820,14.5270 (-0.2820)
9,redundant,0.14366,14.3660,-0.1610,14.3660 (-0.1610)


In [26]:
combined = pd.concat(
    [
        final_behavior_regs[['ABC-Eval', 'R-Squared delta']],
        final_likert_turn_regs[['Likert Turn', 'R-Squared delta']],
        final_likert_dialogue_regs[['Likert Dialogue', 'R-Squared delta']]
    ],
    axis=1)

combined.to_csv('results/paper/incremental_validity.csv', index=False)
combined

,ABC-Eval,R-Squared delta,Likert Turn,R-Squared delta,Likert Dialogue,R-Squared delta
0,antisocial,15.9640 (nan),grammatical,8.5660 (nan),emotional,58.4260 (nan)
1,life info,15.9550 (-0.0090),informative,8.5080 (-0.0580),informative,57.8320 (-0.5940)
2,preference info,15.8500 (-0.1050),engaging,8.2850 (-0.2230),grammatical,56.4390 (-1.3930)
3,commonsense contradiction,15.6920 (-0.1580),emotional,7.7530 (-0.5320),relevant,54.5170 (-1.9220)
4,uninterpretable,15.5030 (-0.1890),consistent,6.8890 (-0.8640),proactive,51.6360 (-2.8810)
5,ignore,15.2500 (-0.2530),proactive,5.7860 (-1.1030),consistent,48.6730 (-2.9630)
6,correct fact,15.1970 (-0.0530),relevant,3.7760 (-2.0100),engaging,38.1030 (-10.5700)
7,topic switch,14.8090 (-0.3880),NaN,NaN,NaN,NaN
8,follow up,14.5270 (-0.2820),NaN,NaN,NaN,NaN
9,redundant,14.3660 (-0.1610),NaN,NaN,NaN,NaN
